### Chat with your LinkedIn Profile


In [ ]:
# From the agents library import necessary components
from dotenv import load_dotenv
from agents import Agent, Runner, trace, SQLiteSession

#### Load environment

In [ ]:
load_dotenv(override=True)

In [ ]:
# Define model
from openai import AsyncOpenAI
from agents import OpenAIChatCompletionsModel
import os

# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using the local LLM server client
# thinking mode: gpt-oss. takes time
model = OpenAIChatCompletionsModel(model = "llama3.2:latest",openai_client= client)

#### Read Linkedin PDF to text

Parsing linkedin PDF to text and creating system prompt

In [ ]:
from pypdf import PdfReader

reader = PdfReader("me.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name="Priyabrata"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

system_prompt

In [ ]:
# Define Agent
# Create an agent using the local model
local_agent = Agent(name="LinkedInAgent",
                    instructions=system_prompt,
                    model=model)

### Gradio chat app with local llm

In [ ]:
session = SQLiteSession("linkedin_agent_session.db")

async def chat(message, history):
    result = await Runner.run(local_agent, message, session=session)
    return result.final_output

import gradio as gr
gr.ChatInterface(
    chat,
    title="Priyabrata LinkedIn Agent",
    description="Chat with Priyabrata's LinkedIn agent."
).launch()
